# GAN

1. [Adathalmaz, tanítóhalmaz előkészítése](#Adathalmaz,-tanítóhalmaz-előkészítése)
2. [Modell-tanítása](#Modell-tanítása)
    1. [Hibafüggvények](#Hibafüggvények)
    2. [Optimalizáló-módszer](#Optimalizáló-módszer)
    3. [Tanítási-lépés](#Tanítási-lépés)
3. [Mode-Collapse-jelensége](#Mode-Collapse-jelensége)
4. [Regularizációs-módszerek](#Regularizációs-módszerek)
    1. [Megfelelő-inicializációs-stratégia](#Megfelelő-inicializációs-stratégia)
    2. [Aktivációs-függvények-és--paramétereiknek-a-helyes-megválasztása](#Aktivációs-függvények-és--paramétereiknek-a-helyes-megválasztása)
    1. [Batch-normalization](#Batch-normalization)
    2. [Spectral-Normalization](#Spectral-Normalization)
    3. [Experience-replay](#Experience-replay)
    4. [Mini-batch-discrimination](#Mini-batch-discrimination)
5. [Data-augmentation---tanítás-kevés-adattal](#Data-augmentation---tanítás-kevés-adattal)

A _Generative Adverserial Network_ (GAN), egy olyan generatív modell, amely nem a megszokott statisztikai alapokon optimalizál, mint például az Autoencoder vagy pixelRNN modellek, hanem játékelméleti megközelítést alkalmaz. (Az generatív modellekről lehetne írni esetleg egy áttekintést, hogy miért éppen a GAN-ra esett a választás mondjuk a VAE-val szemben?)
A tanulás során két neurális hálózat versenyzik egymással: egy _generátor_, amelynek az a szerepe, hogy a tanítómintákhoz hasonló adatot generáljon a bemeneti zajból és egy _diszkriminátor_, amely egy bináris osztályozó, amely a generátor által generált adatot vizsgálja és eldönti, hogy az valódi vagy hamis.
A tanítás során ezen két háló versenyzik egymással, együtt fejlődve.
A generátor bemenete egy zajvektor, amely általában Gauss- vagy egyenletes eloszlásból állítunk elő. Ezt a zajvektort az irodalom látens térnek is nevezi, hiszen a tanítás során a modell megtanulja, hogy ezen többdimenziós tér egyes pontjaira milyen kimenetet generáljon. Vagyis ezzel lényegében kitölti a rendelkezésre álló tér tartományait a megtanult jellegzetességekkel. A generátorral ezen tér bármely pontját mintavételezve a tanítóhalmazhoz hasonló adatokat generálhatunk.

## Adathalmaz, tanítóhalmaz előkészítése
A tanításhoz szükségünk van egy megfelelő adathalmazra. Képek generálásához legegyszerűbb esetben elegendő lehet képek rendezetlen halmaza is és a modellre bízhatjuk, hogy ismerje fel az egyes osztályok jellegzetességeit. Megfelelő regularizációs technikákkal igazán változatos képek generálására lesz képes a betanított modell.

Ha az adathalmaz rendelkezik osztályokkal is, úgy az osztály-címkéket is felhasználhatjuk a GAN hálózatunk tanításához. Így egy plusz bemenet segítségével könnyebben tudunk majd megfelelő képeket előállítani. (Ezt a technikát class-conditioningnak nevezik, de utána kell nézzek)

Egyes adathalmazokhoz igen részletes annotációkat is mellékelnek. A képeken megfigyelhető objektumokat határoló dobozokkal, vagy pixel szinten is jelölik. Így igen részletes információkat kínálnak a kép tartalmáról. Egy igazán hasznos annotációk lehetnek a természetes nyelvű leíró mondatok is, amelyekből általában többet is mellékelnek egy-egy képhez.

Ha kiválasztottuk a modellünkhöz megfelelő adathalmazt, akkor a probléma és a modell bonyolultságától függően elő kell készítenünk az adatokat a tanításhoz.
A GAN esetében a hálózat paraméterei közé tartozik a minibatch elemszáma is. Minibatch alatt a tanítóhalmaz egy bizonyos elemszámú mintát tartalmazó szeletét értjük. A minibatch-ok alkalmazása azért lényeges, hogy egy tanítási lépés során a modell ne lássa a teljes tanítóhalmazt. A hálózat tanításához általában sok mintára van szükségünk, így egy darabban való tanítás igen nehézkes és számításigényes lenne. De nem csupán ez jelenti a fő veszélyt, Ha egy lépésben láthatná a diszkriminátor az összes képet, akkor túlságosan is megnőne a teljesítménye és a generátornak esélye sem lenne felzárkózni. A minibatch-okon való tanítás tehát elengedhetetlen része a tanítási folyamatnak és a megfelelő ütemű tanulás biztosításának.


## Modell tanítása
Ha a legegyszerűbb esetet vizsgáljuk és csak a képek rendezetlen halmazára tanítjuk a modellt, mindenféle kiegészítő információ és annotáció nélkül, akkor a tanítás a következőképpen zajlik.
A továbbiakban a következő jelöléseket használom: legyen $D$ a _diszkriminátor_, $G$ pedig a _generátor_.

### Hibafüggvények
A $G$ és $D$ hibájának számolása a bináris kereszt-entrópián alapszik.
A bináris kereszt-entrópia hibafüggvény a következőképpen írható fel:
$$L(\hat y, y) = y . \log \hat y + (1-y). \log (1 - \hat y)$$
Ahol $\hat y$ a predikció, $y$ pedig a valós címke

**Diszkriminátor hibafüggvénye**

A $G$ generátor egy $z \in \mathbb{R}^n, n \geq 1$ bemeneti zajvektor alapján előállít egy generált adatot $G(z)$.
A $D$ diszkriminátor egy bináris osztályozó, amelynek feladata, hogy az $x$ és $G(z)$ bemeneteit osztályozza.\\
$D(x)$ esetén 1, $D(G(z))$ esetén pedig 0 címkét várunk.\\
A hibafüggvény számolása két lépésben történik a kétféle bemenet miatt:

$D(x)$-re nézve a kereszt-entrópia a következő:
$$L(D(x), 1) = 1.\log D(x) + (1 - 1).\log(1 - D(x))$$
$$L(D(x), 1) = \log D(x)$$
Vagyis jelen esetben $D$-nek a $\log(D(x))$-et kell maximalizálnia.

$D(G(z))$-re nézve a kereszt-entrópia a következő:
$$L(D(G(z)), 0) = 0.\log D(G(z)) + (1 - 0).\log(1 - D(G(z)))$$
$$L(D(G(z)), 0) = \log(1- D(G(z)))$$
Vagyis a $\log(1 - D(G(z)))$-t kell maximalizálnia.

Egyetlen mintára a hibafüggvény a következőképpen néz ki:
$$\max V(D) = \log D(x) + \log(1 - D(G(z))$$

Batch-ra nézve:
$$\max V(D) = \mathbb{E}_{x \sim P(x)} \left[\log D(x) \right] + \mathbb{E}_{z \sim P(z)} \left[\log(1 - D(G(z))) \right]$$

Ahol a $P(x)$ a valószínűségi eloszlása a tanítóhalmaznak, $P(z)$ a valószínűségi eloszlása a $z$ zajvektornak. (látens tér).

**Generátor hibafüggvénye**

A $G$ generátor feladata az, hogy megtévessze a $D$ diszkriminátort azáltal, hogy a tanítóhalmazhoz hasonló adatokat generáljon.
Vagyis a $G$ érdeke az, hogy a $D(G(z))$ 1-es címkét kapjon 0 helyett.

Tehát a bináris keresztentrópia egy mintára:
$$L(D(G(z)), 0) = \log(1 - D(G(z))$$
$D$ minimalizálni kívánja a $D(G(z))$-t, míg a $G$ maximalizálni szándékozik azt.

A $G$ a tanítás során sosem fog valódi adatot látni, de a teljesség kedvééert a hibafüggvénye a következőképpen írható fel (Csak a második kifejezést minimalizálja valójában):
$$\min V(G) = \mathbb{E}_{x \sim P(x)} \left[\log D(x) \right] + \mathbb{E}_{z \sim P(z)} \left[\log(1 - D(G(z))) \right]$$

Vagyis a GAN hálózat tanítása során $D$ és $G$ egy minimax játékot játszanak a $V(G, D)$ értékfüggvénnyel.
$$\min_{G}\max_{D}V(D, G) =  \mathbb{E}_{x \sim P(x)} \left[\log D(x) \right] + \mathbb{E}_{z \sim P(z)} \left[\log(1 - D(G(z))) \right]$$

### Optimalizáló módszer
A Generátorban is és a Diszkriminátoban is az Adam optimalizáló használatos. (Kezdeti paraméterek, learning rate, beta1-2)
Itt vagy egy másik kis fejezetben be lehetne mutatni az optimalizáló módszereket?

### Tanítási lépés
%% GAN 2014 cikkből

A GAN hálózat egy tanítási lépése a következő lépésekből áll:

Legyen $m$ a minibatch elemszáma $m \in \mathbb{N}$

1. Hozzunk létre $m$ darab zajmintát $(z_1, \ldots, z_m)$ gauss eloszlásból $P_g(z)$.
2. A tanítóhalmazból emeljük ki a soronkövetkező $m$ darab tanítómintát (képet), és ezt jelöljük $(x_1, \ldots, x_m)$-el $P_{\text{data}}(x)$
3. Frissítsük a $D$ diszkriminátort a sztochasztikus gradiens emelkedésével (?? tükörfordítás)
$$ \nabla \theta_d \frac{1}{m} \sum_{i=1}^{m} \left[\log D(x_i) + \log(1 - D(G(z_i))) \right]$$
4. Frissítsük a $G$ generátort a sztochasztikus gradiens lejtésével (?? tükörfordítás)
$$ \nabla \theta_d \frac{1}{m} \sum_{i=1}^{m} \log(1 - D(G(z_i)))$$

Természetesen nem egyszerre tanítjuk a GAN részeit. Az eredeti cikkben is javaslatot tesznek arra, hogy a $D$-t esetleg több lépésben is lehetne tanítani, majd a $G$-t egyetlen lépésben frissíteni.
Különböző tanítási stratégiákban ez is egy szabad paraméter lehet. Számomra megfelelő volt az 1:1-es tanítási lépés alkalmazása is... (Esetleg lehetne mérni valahogy, hogy van-e számottevő különbség...)
A tanítás hossza természetesen függ az adathalmaztól és annak méretétől, a minibatch mérettől, a modellben található paraméterektől és az optimalizáló függvénytől.

## Mode-Collapse jelensége
A mode-collapse jelenség akkor figyelhető meg, amikor a Generátor a látens tér bármely pontjára egyetlen egy (vagy ritkán kétféle) képet generál, ezzel minimalizálva a hibafüggvényét.
Egyes cikkekben az összeomlásig tanítják a felvázolt modelleket és az összeomlás előtti állapotokra állítják vissza a hálózat súlyait a használathoz (BigGAN csinált ilyet, meg még más is, de meg kell keresnem.). Az mode collepse elkerülésére és a hálózat kiegészítésére igen hasznos javaslatok jelentek meg. Ezeket külön-külön is kifejtem... (!)

Ide jöhetne egy példa kép is, amikor a sok képből álló minta minden egyes esetre ugyan azt adja vagy valami....

## Regularizációs módszerek
(Még csak a Batch Normalization-t próbáltam az alapokon kívül)

A GAN tanítása során nehézségekbe ütközhetünk. Legrosszabb esetben nem is kezd el konvergálni a generátor kimenete a tanítóminta képeihez, viszont abban az esetben gyanakodhatunk, hogy a modellünk nem lett helyesen felépítve. Amennyiben mégis elkezd fejlődni a modell és nem csak zaj jelenik meg a kimeneten az egyes tanítólépések után, de bizonyos számú epoch után mode collapse lép fel, úgy fontolóra vehetjük a következő regularizációs technikákat.

### Megfelelő inicializációs stratégia


### Aktivációs függvények és  paramétereiknek a helyes megválasztása


### Batch normalization


### Spectral Normalization


### Experience replay


### Mini-batch discrimination


## Data augmentation - tanítás kevés adattal
- karras2020training
- noguchi2019image